### Strategy:

"Find the longest strand of bytes that exists in at least two files."

The problem of option 1, can be reduced to the more commonly known "longest common substring" problem. Basically, given strings x and y, try to find the maximum length of [x_i, x_j] and [y_i, y_j] such that len([x_i, x_j]) == len([x_i, x_j]). i and j being the starting and end position of a substring respectively.

There are a few design questions to tackle first. Primarily, what will be our data types. The bytes given can all be translated into strings or interger values from 0-255. Assuming that memory wouldn't be an issue (which it definitely was), choosing strings severely limits my options of tools available. If I wanted any chance at completing this thing with Python (this wouldn't exactly be fun or difficult on C++/CPython), I had to use a vectorized approach. This then leads me to find a vectorized approach to solving the multi-string LCS problem for arrays of intergers, not strings. It's quite the problem reduction, but at least it worked in the end.

Next, what is the general principle, or solution guideline, that would solve this problem in optimal time-complexity? The most practical answer is probably through suffix arrays. For a quick overview on the suffix array solution for LCS see here: https://www.youtube.com/watch?v=Ic80xQFWevc 

Below I'll give more details on my high-level implementation:

- Start by creating array of 256 arrays (actually lists, no vectorization yet done at this level, only loops)
- To generate all suffixes (or substrings/subsequences of bytes, I use them interchangeably), I can create new arrays for each substring, looping through each original string of bytes, popping a value (at the front, [0]) then adding new array to the array index corresponding with its new starting value. 
- For example, an array start with [32, 45, 12, ...] would be placed into array[31]. 
- Furthermore, add a 3-list containing metadata of the substring containing:
    -   original array number (1 through 10)
    -   length of substring, that corresponds to the substring origin file
    -   index of original string where this suffix begins
- That's confusing, data structure looks like this:

```
main_array = [[[<all subarrays (0) origin file>, <all subarrays (0) length>], [<all sub-arrays starting with 0>]], [[<all subarrays (1) origin file>, <all subarrays (1) length>], [<all sub-arrays starting with 1>]],..., [[<all subarrays (100) origin file>, <all subarrays (100) length>], [<all sub-arrays starting with 100>]], ...,[[<all subarrays (255) origin file>, <all subarrays (255) length>], [<all sub-arrays starting with 255>]]]]
```
- The advantage to this data structure is that it will allow for us to conduct our sort while maintaining metadata about where that substring came from, as well as it's length and original index position (since the length will eventually be transformed so as to have even sized numpy arrays)

- Heuristics play a huge part in final performance of my solution
- For example, because of our meta-data, we can ignore sorting all substrings less than current maximum length
    - exponentially increases runtime and improves memory
- To sort all substrings of a starting number, they first have to be the same size so pad all arrays with 0's at the end until they're length of longest array for that number category 
- I can then sort numpy array by column x amount of times, x being length of arrays in the group. We can find max sub-array length (for say those starting with 165) by:

```pseudocode
# 1 indicates the array of subarrays
# this loops over all substrings beginning with 165
for i in matrix[164][1]:
    temp = []
    temp.append(len(i))
    return max(temp)
 ```  
 - in reality this portion is mostly vectorized  
- I can find longest common prefix by iterating over all pairs in sorted subarrays, comparing if each additional array adds on to total matching length.
- But even better, since my data structure knows what's the last substring for each group, and now they're all sorted, I can work backwards
- Thus, if the longest subarray is stored, I don't have to check many of the (usually) smaller subarrays ahead of it. Granted they're smaller than our max length already, it would just be redundant
- In short, the final game plan is to continually improve stored maximum value, and as it improves, sorts and searches get easier and easier (by weeding out unecessary candidates)
- I suppose this solution isn't ideal. It notably sacrifices sort time for an optimal search time. However, it gives a respectable solution that doesn't use Cython or CPython, and in fact, beating out CPython solution. (And to be fair, the brute force python approach here wouldn't even run)
- The next benefit is storage size with numpy. Each byte is a small 8-bit number, which cuts memory drastically. Something you'd have to find a workaround for even in C++
    - This is, of course assuming we're not running in the worst case scenario, where we can never weed out smaller substrings
    - However, the worse case is extremely rare. Especially since we're re-organizing separate strings, and re-combining them into separate data-structure, odds of worst case is very rare

### For anyone looking at this under a critical eye...
- The last major part to improve is the unlikely worst-case scenario outcome, where sorting matrices becomes exponentially more expensive
- In such cases, the only part to optimize is really just the sort. In my tests, in can take up to 90% of run time when no heuristic is applied
- Obviously, not great, and extremely rare scenario but still worth considering for improvements
- One such improvement could be detection of worst case scenario, where we just work backwards and now it's best case. Just fun ideas

failed ideas (ignore, just left here so I don't try them again in future):
- pandas: I tried using pandas because of it's easy .fillna function which I though would be perfect to create all needed 0's and row shapes, then sort by column from right 
to left. Terrible idea, forgot how slow pandas was in general. Barely runs.
- np.insert origin file and substring size into front of array: Not great, large numbers (string size) requires at least int32, 4x as many bites and slows performance
- apply(): primarily for search. It's better than dataframes, but ideal solutions still require full vectorizations
- np.concatenate, np.hstack, np.zeros, np.isnan, etc. basically anything to quickly pad all vectors. I think I literally must've tried everything. Final padding solution here is blazing fast, by far the best
- numba: This one is really unfortunate. I was really looking forward to numba working, but a) they seem to be going through updates/feature testing so some things are quite broken and b) they still don't work with certain numpy functions like np.pad and np.lexsort. The improvements with numba but also with those replacements would be questionable. Something I might explore in the future, but otherwise I'm content at this point
- no heuristic!! pound for pound, my solution doesn't quite hold up to CPython's (although to be fair they also use heuristics like automatic junk, https://docs.python.org/3/library/difflib.html). However, because my heuristic is all-inclusive (at least I believe), there's no point to not using it. Don't sort arrays that are too small to possibly be new max. Simple and elegant. It was one of my last optimizing ideas, really wish it came to me earlier.


In [1]:
# Load libraries and data. ~1 sec run-time

from pathlib import Path
import numpy as np
import pandas as pd
from numba import njit, jit
from difflib import SequenceMatcher 
import time
from numba.typed import List
from operator import itemgetter

# normally I wouldn't write such redundant code but it's only ten files so eh
with open("data/sample.1", "rb") as file1, open("data/sample.2", "rb") as file2, open("data/sample.3", "rb") as file3, open("data/sample.4", "rb") as file4, open("data/sample.5", "rb") as file5, open("data/sample.6", "rb") as file6, open("data/sample.7", "rb") as file7, open("data/sample.8", "rb") as file8, open("data/sample.9", "rb") as file9, open("data/sample.10", "rb") as file10:
    file_1 = file1.read()
    file_2 = file2.read()
    file_3 = file3.read()
    file_4 = file4.read()
    file_5 = file5.read()
    file_6 = file6.read()
    file_7 = file7.read()
    file_8 = file8.read()
    file_9 = file9.read()
    file_10 = file10.read()
# we can see that the length of encoded array matches exactly the file size. E.g, sample.1 is 17.408 kb
# that's exact same length of encoded_array_1 (17408)
encoded_array_1 = bytearray(file_1)
encoded_array_2 = bytearray(file_2)
encoded_array_3 = bytearray(file_3)
encoded_array_4 = bytearray(file_4)
encoded_array_5 = bytearray(file_5)
encoded_array_6 = bytearray(file_6)
encoded_array_7 = bytearray(file_7)
encoded_array_8 = bytearray(file_8)
encoded_array_9 = bytearray(file_9)
encoded_array_10 = bytearray(file_10)

encoded_arrays = [encoded_array_1, encoded_array_2, encoded_array_3, encoded_array_4,
                 encoded_array_5, encoded_array_6, encoded_array_7, encoded_array_8,
                  encoded_array_9, encoded_array_10
                 ]
print("Size of sample.1 in bytes: ", Path('data/sample.1').stat().st_size)

for val, i in enumerate(encoded_arrays):
    print("Length of sample." + str(val+1) + " byte array: ", len(i))

# helper function that pads all rows so they are same length as longest row
def pad_numpy_matrix(matrix):
    
    l = [len(a) for a in matrix]
    max_len = max(l)

    # np.pad is much more space efficient than alternatives
    c = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0) for a in matrix])
    # c = np.asarray([np.hstack((a, np.zeros(max_len-a.shape[0], dtype = np.int8))) for a in matrix])
    return c

Size of sample.1 in bytes:  17408
Length of sample.1 byte array:  17408
Length of sample.2 byte array:  30720
Length of sample.3 byte array:  45056
Length of sample.4 byte array:  30720
Length of sample.5 byte array:  23552
Length of sample.6 byte array:  27648
Length of sample.7 byte array:  21504
Length of sample.8 byte array:  20480
Length of sample.9 byte array:  13312
Length of sample.10 byte array:  14336


In [2]:
# Solution using CPython difflib
# ~30s for bytes
def longest_Substring(s1,s2): 
     seq_match = SequenceMatcher(None,s1,s2) 
     match = seq_match.find_longest_match(0, len(s1), 0, len(s2)) 

     # return the longest substring 
     if (match.size!=0): 
          return (s1[match.a: match.a + match.size])  
     else: 
          return ('Longest common sub-string not present') 

def check_all(arrays):
     
     # https://github.com/python/cpython/blob/0daf72194bd4e31de7f12020685bb39a14d6f45e/Lib/difflib.py#L305
     array_copy = arrays.copy()
     maxes = []
     count = 0

     # no need to compare the same two arrays twice, 1+2+3+...+10 = 45 comparisons only
     for i in range(len(array_copy)):
          a = array_copy[0]
          del array_copy[0]
          count += 1
          next_count = 0
          for j in range(len(array_copy)):
               next_count += 1
               add = count+next_count
               longest = longest_Substring(a, array_copy[j])
               maxes.append((count,(count+next_count),len(longest)))
          
     return maxes

str_arrays = []
bytes_arrays = []
for i in encoded_arrays:
     str_arrays.append(str(bytes(i)))
     bytes_arrays.append(bytes(i))
answer_str = check_all(str_arrays)
str_pair = max(answer_str,key=itemgetter(2))[0:2]
str_max_length = max(answer_str,key=itemgetter(2))[2]

print("The longest common substring(str) length is: ", str_max_length)
print("The longest common substring(str) origin files are: ", str_pair, '\n')

start = time.time()
answer_bytes = check_all(bytes_arrays)
end = time.time()
bytes_pair = max(answer_bytes,key=itemgetter(2))[0:2]
bytes_max_length = max(answer_bytes,key=itemgetter(2))[2]
cpython_runtime = end-start
print("The longest common substring(bytes) length is: ", bytes_max_length)
print("The longest common substring(bytes) origin files are: ", bytes_pair, '\n')
print("CPython runtime (bytes): ", cpython_runtime)

The longest common substring(str) length is:  41243
The longest common substring(str) origin files are:  (2, 3) 

The longest common substring(bytes) length is:  27648
The longest common substring(bytes) origin files are:  (2, 3) 

CPython runtime (bytes):  37.119706869125366


In [3]:
# Testing CPython Solutions to show that a string of max length does actually exist. If so, then new solution only has to match CPython solution

s1 = np.array(encoded_arrays[1])
s2 = np.array(encoded_arrays[2])
# print(len(s1), len(s2))

x = 27648
seq_match = SequenceMatcher(None,s1,s2) 
match = seq_match.find_longest_match(0, len(s1), 0, len(s2))
print("Is CPython solution correct? ", np.all(s1[match[0]:match[0]+x]==s2[match[1]:match[1]+x]))

Is CPython solution correct?  True


In [5]:
# let's make everything a numpy array of arrays. Looping to append to 
# np arrays is actually very inefficient so convert at end instead
def create_np_array(list_arrays):
    allArrays = []
    for i in list_arrays:
        allArrays.append(np.array(i))
    # test that everything went ok
    #print(len(allArrays), len(allArrays[0])==len(allArrays[1]), type(allArrays[0]))
    main_array = []
    for i in range(256):
        main_array.append([[], []])

    # repeat this for each file (10x)
    for val, i in enumerate(allArrays):
        new_i = i
        # add each possible array
        for j in range(len(i)):
            head = new_i[0]
            # each array in 256 main arrays will have the original file in position 0 followed by actual substring content
            # to_append = np.concatenate(([val], new_i))
            a = len(new_i)
            main_array[head][0].append([val+1, a, j])
            main_array[head][1].append(new_i)
            new_i = new_i[1:]
    print('complete enumeration')
    return(main_array)

def sort_matrix(main_array, global_max):
    y = np.array(main_array[0])
    lengths = y[:,1]
    max_length = np.max(lengths)

    # if the maximum length of all strings in this current matrix is less than current max substring, no need to bother. Skip this matrix
    if max_length>global_max[2]:
        
        # one of the most effective things by far. There's no need to assess strings whose original lengths < current max substring
        # so let's remove them, and only sort the rows that are larger. Easy matrix transformations
        use_rows = lengths > global_max[2]
        temp_final = np.array(main_array[1])[use_rows]
        keys_final = y[use_rows]
        #b = time.time()
        #print('make np: ', b-a)
        # make all arrays same size. Pad with 0's so sortable
        temp_padded = pad_numpy_matrix(temp_final)

        #c = time.time()
        #print('pad rows: ', c-a)

        # np.lexsort each matrix, returns the final order of rows for sort
        # by far the biggest eater of time here. ~80-95%. If we can cut this down, drastic improvement
        sort_order = np.lexsort(np.fliplr(temp_padded).T)
        #sort_order = -temp_padded[:,:].argsort(axis=0)[0]

        #d = time.time()
        #print('sort: ', d-a)
        # we also want keys to follow new order. This way we can sort rows independent of position, size (keys)
        sorted_keys = keys_final[sort_order]
        temp_sorted = temp_padded[sort_order]
        #e = time.time()
        #print('append: ', e-a)
        return [sorted_keys, temp_sorted]
    else:
        return None

def max_suffix(array, global_max, debug=False):

    # [[meta], [matrix]]
    # global_max = (x, y, length) . Both original file locations and length of current max substring
    meta = array[0]
    matrix = array[1]
    new_max = global_max
    # i is each 256 [[],[]]
    # if there isn't more than one substring beginning with that number no need to check
    if len(meta) >1:
        for j in range(1, len(matrix)):
            a = j
            b = j+1
            if meta[-a][1] > new_max[2]:
                if debug:
                    print("a and b", a, b)
                    print("meta", meta[-a][0], meta[-b][0])
                # we want the first row working backwards that's not from same origin file
                
                if meta[-a][0] != meta[-b][0]:
                    # np.ndarray
                    # b < a in length
                    a_array = np.array(matrix[-a])
                    b_array = np.array(matrix[-b])
                    if debug:
                        print("a", a_array)
                        print("b", b_array)
                    c = a_array==b_array
                    #non_zero_array = np.nonzero(c == False)[0]
                    if debug:
                        print(c) #non_zero_array)
                    if np.sum(c) > 0:
                        if np.all(c):
                            LCS_length = meta[-b][1]
                        else:
                            LCS_length = np.nonzero(c == False)[0][0]
                        if debug:
                            print(LCS_length)
                        if LCS_length > new_max[2]:
                            # we need the original string size precisely here. If two substrings are perfect match, we don't want all appended 0's there as well
                            # only append actual length of substring added, not padded length
                            new_max = (meta[-a][0], meta[-b][0], LCS_length, (meta[-a][2], meta[-b][2]))
                            #print("updated", new_max)
                else:
                    pass
    return new_max

def loop_matrix(matrix, debug=False):
    
    LCS_matrix = create_np_array(matrix)
    global_max = (0,0,0,0,0)
    #start = time.time()
    for i in range(len(LCS_matrix)):
        print("Completed search through all sub-sequences beginning with: ", i, global_max)
        if len(LCS_matrix[i][0]) > 1:
            temp = sort_matrix(LCS_matrix[i], global_max)
        else:
            temp = None
        #end = time.time()
        #print("finished sorting in: ", end-start)
        if temp:
            global_max = max_suffix(temp, global_max, debug)
        #end = time.time()
        #print("finished group: ", i, " in ", end-start)
    return global_max

# I spent hours debugging a simple mistake... This is why testing is important. If you'd like the test, simply set debug=True. Plenty of debugging comments as well
test = [[1,3,3,4,5,6,7],
        [3,1,4,2,7,3,4,5,6,7],
        [1,5,6,7,6,7,4,2,7,3,4,3,1,4,2,7,3,4,5,6,7],
        [1,4,8,7,8,9,10],
        [3,6,1,4,2,7],
        [9,8,4,0,3,4,5]]

start = time.time()
lcs_ = loop_matrix(encoded_arrays, False)
end = time.time()
my_runtime = end-start

print("The longest common substring(bytes) length is: ", lcs_[2])
print("The longest common substring(bytes) origin files are: ", lcs_[0:2])
print("Strings begin in positions: ", lcs_[3])
# Yup it works
print("Is final answer correct? ", lcs_[2]==bytes_max_length)
print("Is my version faster? ", my_runtime<cpython_runtime, my_runtime)

complete enumeration
Completed search through all sub-sequences beginning with:  0 (0, 0, 0, 0, 0)


<ipython-input-5-a08a16c6e6fe>:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp_final = np.array(main_array[1])[use_rows]


Completed search through all sub-sequences beginning with:  1 (3, 2, 27515, (17541, 3205))
Completed search through all sub-sequences beginning with:  2 (3, 2, 27515, (17541, 3205))
Completed search through all sub-sequences beginning with:  3 (3, 2, 27515, (17541, 3205))
Completed search through all sub-sequences beginning with:  4 (3, 2, 27636, (17420, 3084))
Completed search through all sub-sequences beginning with:  5 (3, 2, 27636, (17420, 3084))
Completed search through all sub-sequences beginning with:  6 (3, 2, 27640, (17416, 3080))
Completed search through all sub-sequences beginning with:  7 (3, 2, 27640, (17416, 3080))
Completed search through all sub-sequences beginning with:  8 (3, 2, 27640, (17416, 3080))
Completed search through all sub-sequences beginning with:  9 (3, 2, 27640, (17416, 3080))
Completed search through all sub-sequences beginning with:  10 (3, 2, 27640, (17416, 3080))
Completed search through all sub-sequences beginning with:  11 (3, 2, 27640, (17416, 3080